In [2]:
import re
from typing import List, Union
# Python内置模块，用于格式化和包装文本
import textwrap
import time

from langchain.agents import (
    Tool,          # 可用工具
    AgentExecutor, # Agent执行
    LLMSingleActionAgent, # 定义Agent
    AgentOutputParser, # 输出结果解析
)
from langchain.prompts import StringPromptTemplate
# LLMChain，包含一个PromptTemplate和一个LLM
from langchain_community.llms import Tongyi  # 导入通义千问Tongyi模型
from langchain import LLMChain
# Agent执行，Agent结束
from langchain.schema import AgentAction, AgentFinish
# PromptTemplate: 管理LLMs的Prompts
from langchain.prompts import PromptTemplate
from langchain.llms.base import BaseLLM

# 定义了LLM的Prompt Template
CONTEXT_QA_TMPL = """
根据以下提供的信息，回答用户的问题
信息：{context}

问题：{query}
"""
CONTEXT_QA_PROMPT = PromptTemplate(
    input_variables=["query", "context"],
    template=CONTEXT_QA_TMPL,
)

# 输出结果显示，每行最多60字符，每个字符显示停留0.1秒（动态显示效果）
def output_response(response: str) -> None:
    if not response:
        exit(0)
    # 每行最多60个字符
    for line in textwrap.wrap(response, width=60):
        for word in line.split():
            for char in word:
                print(char, end="", flush=True)
                time.sleep(0.1)  # Add a delay of 0.1 seconds between each character
            print(" ", end="", flush=True)  # Add a space between each word
        print()  # Move to the next line after each line is printed
    # 遇到这里，这个问题的回答就结束了
    print("----------------------------------------------------------------")

# 模拟公司产品和公司介绍的数据源
class TeslaDataSource:
    def __init__(self, llm: BaseLLM):
        self.llm = llm

    # 工具1：产品描述
    def find_product_description(self, product_name: str) -> str:
        """模拟公司产品的数据库"""
        product_info = {
            "Model 3": "具有简洁、动感的外观设计，流线型车身和现代化前脸。定价23.19-33.19万",
            "Model Y": "在外观上与Model 3相似，但采用了更高的车身和更大的后备箱空间。定价26.39-36.39万",
            "Model X": "拥有独特的翅子门设计和更加大胆的外观风格。定价89.89-105.89万",
        }
        # 基于产品名称 => 产品描述
        return product_info.get(product_name, "没有找到这个产品")

    # 工具2：公司介绍
    def find_company_info(self, query: str) -> str:
        """模拟公司介绍文档数据库，让llm根据信息回答问题"""
        context = """
        特斯拉最知名的产品是电动汽车，其中包括Model S、Model 3、Model X和Model Y等多款车型。
        特斯拉以其技术创新、高性能和领先的自动驾驶技术而闻名。公司不断推动自动驾驶技术的研发，并在车辆中引入了各种驾驶辅助功能，如自动紧急制动、自适应巡航控制和车道保持辅助等。
        """
        # prompt模板 = 上下文context + 用户的query
        prompt = CONTEXT_QA_PROMPT.format(query=query, context=context)
        # 使用LLM进行推理
        return self.llm(prompt)


AGENT_TMPL = """按照给定的格式回答以下问题。你可以使用下面这些工具：

{tools}

回答时需要遵循以下用---括起来的格式：

---
Question: 我需要回答的问题
Thought: 回答这个上述我需要做些什么
Action: "{tool_names}" 中的一个工具名
Action Input: 选择这个工具所需要的输入
Observation: 选择这个工具返回的结果
...（这个 思考/行动/行动输入/观察 可以重复N次）
Thought: 我现在知道最终答案
Final Answer: 原始输入问题的最终答案
---

现在开始回答，记得在给出最终答案前，需要按照指定格式进行一步一步的推理。

Question: {input}
{agent_scratchpad}
"""


class CustomPromptTemplate(StringPromptTemplate):
    template: str  # 标准模板
    tools: List[Tool]  # 可使用工具集合

    def format(self, **kwargs) -> str:
        """
        按照定义的 template，将需要的值都填写进去。
        Returns:
            str: 填充好后的 template。
        """
        # 取出中间步骤并进行执行
        intermediate_steps = kwargs.pop("intermediate_steps")  
        print('intermediate_steps=', intermediate_steps)
        print('='*30)
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # 记录下当前想法 => 赋值给agent_scratchpad
        kwargs["agent_scratchpad"] = thoughts  
        # 枚举所有可使用的工具名+工具描述
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in self.tools]
        )  
        # 枚举所有的工具名称
        kwargs["tool_names"] = ", ".join(
            [tool.name for tool in self.tools]
        )  
        cur_prompt = self.template.format(**kwargs)
        #print(cur_prompt)
        return cur_prompt

"""
    对Agent返回结果进行解析，有两种可能：
    1）还在思考中 AgentAction
    2）找到了答案 AgentFinal
"""
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        """
        解析 llm 的输出，根据输出文本找到需要执行的决策。
        Args:
            llm_output (str): _description_
        Raises:
            ValueError: _description_
        Returns:
            Union[AgentAction, AgentFinish]: _description_
        """
        # 如果句子中包含 Final Answer 则代表已经完成
        if "Final Answer:" in llm_output:  
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )

        # 需要进行 AgentAction
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"  # 解析 action_input 和 action
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Agent执行
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

# 设置通义千问API密钥
DASHSCOPE_API_KEY = 'sk-882e296067b744289acf27e6e20f3ec0'

if __name__ == "__main__":
    # 定义LLM
    llm = Tongyi(model_name="qwen-turbo", dashscope_api_key=DASHSCOPE_API_KEY)  # 使用通义千问qwen-turbo模型
    # 自有数据
    tesla_data_source = TeslaDataSource(llm)
    # 定义的Tools
    tools = [
        Tool(
            name="查询产品名称",
            func=tesla_data_source.find_product_description,
            description="通过产品名称找到产品描述时用的工具，输入的是产品名称",
        ),
        Tool(
            name="公司相关信息",
            func=tesla_data_source.find_company_info,
            description="当用户询问公司相关的问题，可以通过这个工具了解公司信息",
        ),
    ]
    
    # 用户定义的模板
    agent_prompt = CustomPromptTemplate(
        template=AGENT_TMPL,
        tools=tools,
        input_variables=["input", "intermediate_steps"],
    )
    # Agent返回结果解析
    output_parser = CustomOutputParser()
    # 最常用的Chain, 由LLM + PromptTemplate组成
    llm_chain = LLMChain(llm=llm, prompt=agent_prompt)
    # 定义的工具名称
    tool_names = [tool.name for tool in tools]
    # 定义Agent = llm_chain + output_parser + tools_names
    agent = LLMSingleActionAgent(
        llm_chain=llm_chain,
        output_parser=output_parser,
        stop=["\nObservation:"],
        allowed_tools=tool_names,
    )
    # 定义Agent执行器 = Agent + Tools
    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent, tools=tools, verbose=True
    )

    # 主过程：可以一直提问下去，直到Ctrl+C
    while True:
        try:
            user_input = input("请输入您的问题：")
            response = agent_executor.run(user_input)
            output_response(response)
        except KeyboardInterrupt:
            break

请输入您的问题： Model 3怎么样




> Entering new AgentExecutor chain...
intermediate_steps= []
---
Question: Model 3怎么样  
Thought: 用户询问的是关于Model 3的信息，我需要先查询这款产品的具体描述。  
Action: 查询产品名称  
Action Input: Model 3  

Observation:具有简洁、动感的外观设计，流线型车身和现代化前脸。定价23.19-33.19万
intermediate_steps= [(AgentAction(tool='查询产品名称', tool_input='Model 3', log='---\nQuestion: Model 3怎么样  \nThought: 用户询问的是关于Model 3的信息，我需要先查询这款产品的具体描述。  \nAction: 查询产品名称  \nAction Input: Model 3  '), '具有简洁、动感的外观设计，流线型车身和现代化前脸。定价23.19-33.19万')]
我现在知道Model 3的产品描述，可以据此回答用户的问题。  
Final Answer: Model 3是一款具有简洁、动感外观设计的汽车，其流线型车身和现代化前脸非常吸引人。它的价格区间为23.19万至33.19万。总体来说，Model 3是一款兼具美观与实用性的优质选择。

> Finished chain.
Model 3是一款具有简洁、动感外观设计的汽车，其流线型车身和现代化前脸非常吸引人。它的价格区间为23.19万至33. 
19万。总体来说，Model 3是一款兼具美观与实用性的优质选择。 
----------------------------------------------------------------


请输入您的问题： 上汽大众是怎样的公司




> Entering new AgentExecutor chain...
intermediate_steps= []
---
Question: 上汽大众是怎样的公司  
Thought: 要回答这个问题，我需要了解上汽大众的公司相关信息，例如其业务范围、成立时间、主要产品等。  
Action: 公司相关信息  
Action Input: 上汽大众  


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11124\4265487943.py:76: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.llm(prompt)




Observation:用户提到的问题是“上汽大众”，但根据提供的信息，内容主要是关于特斯拉及其产品和技术的介绍。如果问题是想对比特斯拉与上汽大众，或者询问上汽大众的相关信息，则需要更具体的描述。

以下是一些可能的方向供参考：
1. **上汽大众简介**：  
   上汽大众是由中国上汽集团和德国大众汽车公司合资成立的一家汽车制造企业，主要生产和销售大众品牌的车型。其产品线覆盖了从小型车到SUV、MPV等多个细分市场，例如朗逸、帕萨特、途观等。

2. **与特斯拉的对比**：  
   如果您希望了解上汽大众与特斯拉的区别，可以从以下几个方面进行比较：
   - **品牌定位**：特斯拉专注于电动汽车和新能源技术，而上汽大众的产品线包括传统燃油车、混合动力车以及纯电动车。
   - **核心技术**：特斯拉以自动驾驶技术和高性能电动动力系统著称，而上汽大众则依托大众集团的技术平台，在燃油经济性和可靠性方面表现突出。
   - **市场策略**：特斯拉在全球范围内推广纯电动车型，而上汽大众更多地针对中国市场的需求，推出适合本地消费者的车型。

如果您有更具体的问题，请进一步说明，我会为您提供更详细的解答！
intermediate_steps= [(AgentAction(tool='公司相关信息', tool_input='上汽大众  \n', log='---\nQuestion: 上汽大众是怎样的公司  \nThought: 要回答这个问题，我需要了解上汽大众的公司相关信息，例如其业务范围、成立时间、主要产品等。  \nAction: 公司相关信息  \nAction Input: 上汽大众  \n'), '用户提到的问题是“上汽大众”，但根据提供的信息，内容主要是关于特斯拉及其产品和技术的介绍。如果问题是想对比特斯拉与上汽大众，或者询问上汽大众的相关信息，则需要更具体的描述。\n\n以下是一些可能的方向供参考：\n1. **上汽大众简介**：  \n   上汽大众是由中国上汽集团和德国大众汽车公司合资成立的一家汽车制造企业，主要生产和销售大众品牌的车型。其产品线覆盖了从小型车到SUV、MPV等多个细分市场，例如朗逸、帕萨特、途观等。\n\n2. **与特斯拉的对比**：  \n   如果您希望了解上汽大众与特斯拉的区别，可以从以下几个方面进行比较：\n

请输入您的问题： /




> Entering new AgentExecutor chain...
intermediate_steps= []


ValueError: Could not parse LLM output: `请提供一个问题，以便我按照指定格式进行一步一步的推理并给出最终答案。`

In [3]:
import os
#DASHSCOPE_API_KEY = 'sk-882e296067b744289acf27e6e20f3ec0'
os.getenv("DASHSCOPE_API_KEY")

'sk-dd7ae33a0056483a82660b9392f4eedc'